In [143]:
import numpy as np
import pandas as pd
from difflib import SequenceMatcher
from functools import partial

In [184]:
df = pd.read_csv('/Users/patricknaylor/Desktop/kaggle/kaggle-NOVO/data/train.csv')
corr = pd.read_csv('/Users/patricknaylor/Desktop/kaggle/kaggle-NOVO/data/train_updates_20220929.csv')
corr_ids = list(corr['seq_id'])
df = df[~df['seq_id'].isin(corr_ids)]

In [190]:
wild_type = 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'
print(df.head())
print(len(wild_type))

   seq_id                                   protein_sequence   pH  \
0       0  AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...  7.0   
1       1  AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...  7.0   
2       2  AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...  7.0   
3       3  AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...  7.0   
4       4  AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...  7.0   

                         data_source    tm  first1  last1  first3     last3  \
0  doi.org/10.1038/s41592-020-0801-4  75.7     0.0    0.0     0.0  0.333333   
1  doi.org/10.1038/s41592-020-0801-4  50.5     0.0    0.0     0.0  0.000000   
2  doi.org/10.1038/s41592-020-0801-4  40.5     0.0    0.0     0.0  0.000000   
3  doi.org/10.1038/s41592-020-0801-4  47.2     0.0    0.0     0.0  0.333333   
4  doi.org/10.1038/s41592-020-0801-4  49.5     0.0    0.0     0.0  0.000000   

   first5  last5   first15    last15   first30    last30  sim_ratio   seq_len  
0     0.0    0.2  0.133333  0.

In [186]:
def similar(s, c, w, b, side=None): 
    if side == 'start':
        return SequenceMatcher(None, s[c][:b], w).ratio()
    elif side == 'end':
        return SequenceMatcher(None, s[c][-b:], w).ratio()
    elif side == None:
        return SequenceMatcher(None, s[c][:], w).ratio()
    else:
        raise ValueError('side must be `start`, `end`, or None')
        
def match_columns(wild, train):
    bounds = [1, 3, 5, 15, 30]
    for bound in bounds:
        start_wild = wild[:bound]
        end_wild = wild[-bound:]
        train[f'first{bound}'] = train.apply(partial(similar, c='protein_sequence', w=start_wild, b=bound, side='start'), axis=1)
        train[f'last{bound}'] = train.apply(partial(similar, c='protein_sequence', w=start_wild, b=bound, side='end'), axis=1)
    train['sim_ratio'] = train.apply(partial(similar, c='protein_sequence', w=start_wild, b=bound), axis=1)
    return train
        
        

In [187]:
df = match_columns(wild_type, df)

In [188]:
df['seq_len'] = df['protein_sequence'].str.len()
df['seq_len'] = df['seq_len']/df['seq_len'].max()

In [193]:
df['rel_stab'] = df['tm']/df['tm'].max()


0        0.582308
1        0.388462
2        0.311538
3        0.363077
4        0.380769
           ...   
31385    0.398462
31386    0.286154
31387    0.496923
31388    0.390000
31389    0.289231
Name: rel_stab, Length: 28956, dtype: float64
